In [1]:
!pip install openai==0.28
!pip install -U datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 5.6 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.95.1
    Uninstalling openai-1.95.1:
      Successfully uninstalled openai-1.95.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.8/494.8 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 12.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+

In [2]:
import re
import pandas as pd
import openai
from datasets import load_dataset
from huggingface_hub import login

In [3]:
login()

In [4]:
openai.api_key = 'Your-api'
# comment the below line if you use original OPENAI api
openai.api_base = 'https://api.avalai.ir/v1'

In [6]:
prompt = f'''
A multiple-choice medical question, accompanied by its Persian translation, is provided to you. Your task is to rate the translation on a scale of 0 to 5, where 5 indicates a good translation and 0 indicates a bad one, based on the given evaluation criteria. Please provide your rating as a number only, without any additional explanation
(you should return only a number)

# Evaluation Criteria
- Medical terms have been translated correctly.

# Original Question
Question: {{question}}
Option A: {{optiona}}
Option B: {{optionb}}
Option C: {{optionc}}
Option D: {{optiond}}

# Question translation in Persian
Question: {{question_fa}}
Option A: {{optiona_fa}}
Option B: {{optionb_fa}}
Option C: {{optionc_fa}}
Option D: {{optiond_fa}}
'''

In [14]:
def append_record_to_excel(file_path,id,rate):
        new_record = {
            'id': id,
            'gpt-4.1-mini rate': rate
        }
        new_record_df = pd.DataFrame([new_record])
        try:
            existing_df = pd.read_excel(file_path)
            updated_df = pd.concat([existing_df, new_record_df], ignore_index=True)
        except FileNotFoundError:
            updated_df = new_record_df

        updated_df.to_excel(file_path, index=False)

def get_judge_answer(prompt,m):
    response = openai.ChatCompletion.create(
        model=m,
        messages=[
            {"role": "user", "content": prompt}
        ],
        temperature = 0.0
    )
    result = response['choices'][0]['message']['content'].strip().lower()
    return result
def get_original_question(tid,i,interval):
    for x in range(-interval,interval):
        if i+x < 0 :
           continue
        oid = dataset[i+x]['id']
        if oid==tid :
           return dataset[i+x]
    return None

In [8]:
dataset = load_dataset('mehrdadgh/medmcqa', split='train')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/727 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/85.5M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/182822 [00:00<?, ? examples/s]

In [ ]:
i = 0
bgn = 0
file_num = 0
df = pd.read_excel('./translated_qas_part1.xlsx')
for row in df.iterrows():
    tid = row[1]['id']
    original_mcq = get_original_question(tid,i,10)
    if original_mcq == None:
       raise Exception("question not found!")

    o_question = row[1]['Question']
    o_option1 =  row[1]['Option1']
    o_option2 =  row[1]['Option2']
    o_option3 =  row[1]['Option3']
    o_option4 =  row[1]['Option4']

    t_question = original_mcq['question']
    t_option1 =  original_mcq['opa']
    t_option2 =  original_mcq['opb']
    t_option3 =  original_mcq['opc']
    t_option4 =  original_mcq['opd']

    p = prompt.format(question=o_question,optiona=o_option1,optionb=o_option2,
                      optionc=o_option3,optiond=o_option4,question_fa=t_question,
                      optiona_fa=t_option1,optionb_fa=t_option2,optionc_fa=t_option3,
                      optiond_fa=t_option4)

    judgement = get_judge_answer(p,"gpt-4.1-mini")
    append_record_to_excel('./judgement_part1.xlsx',tid,judgement)

    print(i)
    print('==================================================================================')

    i += 1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27